# Model evaluation and visualization

In [1]:
import pandas as pd
from transformers import XLMRobertaModel, XLMRobertaTokenizer, AdamW
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
val_data = pd.read_csv("/content/drive/MyDrive/Icon Conference/Data/Cleaned_Val_Task_A.csv")
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

MAX_LEN = 128

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [3]:
class FauxHateDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row['Tweet']
        label_faux = row['Hate']
        label_hate = row['Fake']

        # Tokenize the text
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels_faux': torch.tensor(label_faux, dtype=torch.long),
            'labels_hate': torch.tensor(label_hate, dtype=torch.long)
        }

In [4]:
val_dataset = FauxHateDataset(val_data, tokenizer, MAX_LEN)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [5]:
class FauxHateDetector(nn.Module):
    def __init__(self, model_name='xlm-roberta-base', num_labels_task1=2, num_labels_task2=2):
        super(FauxHateDetector, self).__init__()
        self.model = XLMRobertaModel.from_pretrained(model_name)

        # Separate classification heads for 'faux' and 'hate'
        self.classifier_faux = nn.Linear(self.model.config.hidden_size, num_labels_task1)
        self.classifier_hate = nn.Linear(self.model.config.hidden_size, num_labels_task2)

    def forward(self, input_ids, attention_mask):
        # Get the embeddings from the base XLM-RoBERTa model
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # CLS token embedding

        # Task-specific heads
        faux_logits = self.classifier_faux(cls_output)
        hate_logits = self.classifier_hate(cls_output)

        return faux_logits, hate_logits

In [7]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the best model
best_model = FauxHateDetector().to(device)
best_model.load_state_dict(torch.load("/content/drive/MyDrive/Icon Conference/multitaskXLMRoBERTa/best_model.pth", map_location=torch.device(device)))
best_model.eval()

#criterion, optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(params =best_model.parameters(), lr=2e-5)

# Evaluate and visualize on the test set
def evaluate_and_visualize_test(model, dataloader, criterion, device):
    model.eval()
    all_labels_faux, all_preds_faux = [], []
    all_labels_hate, all_preds_hate = [], []
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels_faux = batch['labels_faux'].to(device)
            labels_hate = batch['labels_hate'].to(device)

            # Forward pass
            faux_logits, hate_logits = model(input_ids, attention_mask)

            # Calculate loss
            loss_faux = criterion(faux_logits, labels_faux)
            loss_hate = criterion(hate_logits, labels_hate)
            total_loss += (loss_faux + loss_hate).item()

            # Store predictions and labels
            all_labels_faux.extend(labels_faux.cpu().numpy())
            all_preds_faux.extend(torch.argmax(faux_logits, dim=1).cpu().numpy())
            all_labels_hate.extend(labels_hate.cpu().numpy())
            all_preds_hate.extend(torch.argmax(hate_logits, dim=1).cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    print(f"Test Loss: {avg_loss:.4f}")

    # Print classification reports
    print("Faux Detection Report:\n", classification_report(all_labels_faux, all_preds_faux))
    print("Hate Detection Report:\n", classification_report(all_labels_hate, all_preds_hate))

    # Confusion matrices
    conf_matrix_faux = confusion_matrix(all_labels_faux, all_preds_faux)
    conf_matrix_hate = confusion_matrix(all_labels_hate, all_preds_hate)

    # Plot confusion matrices
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    sns.heatmap(conf_matrix_faux, annot=True, fmt='d', cmap='Blues', ax=axes[0])
    axes[0].set_title("Confusion Matrix for Faux Detection")
    axes[0].set_xlabel("Predicted Labels")
    axes[0].set_ylabel("True Labels")

    sns.heatmap(conf_matrix_hate, annot=True, fmt='d', cmap='Reds', ax=axes[1])
    axes[1].set_title("Confusion Matrix for Hate Detection")
    axes[1].set_xlabel("Predicted Labels")
    axes[1].set_ylabel("True Labels")

    plt.show()

# Assuming `test_dataloader` is already created for test data
evaluate_and_visualize_test(best_model, val_dataloader, criterion, device)


<ipython-input-7-bc86dbe10ff3>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load("/content/drive/MyDrive/Icon Conference/multitaskXLMRoB

KeyError: 'Hate'